In [17]:
pip install clipspy

Note: you may need to restart the kernel to use updated packages.


# Python Function

In [1]:
import tkinter as tk
import clips

def evaluate_and_display():
    # Retract existing disorder facts
    for fact in list(env.facts()):
        if 'disorder' in str(fact):
            fact.retract() 

    # Evaluate the responses for each disorder
    eat_disorder_score = sum(var.get() for var in eat_disorder_responses.values())
    sleep_disorder_score = sum(var.get() for var in sleep_disorder_responses.values())
    anxiety_score = sum(var.get() for var in anxiety_responses.values())
    depression_score = sum(var.get() for var in depression_responses.values())

    # Calculate the total yes count
    total_yes_count = eat_disorder_score + sleep_disorder_score + anxiety_score + depression_score
    env.assert_string(f"(yes-count (value {total_yes_count}))")

    # Assert the scores for each disorder and the total yes count in CLIPS
    env.assert_string(f"(disorder (type \"eat\") (score {eat_disorder_score}))")
    env.assert_string(f"(disorder (type \"sleep\") (score {sleep_disorder_score}))")
    env.assert_string(f"(disorder (type \"anxiety\") (score {anxiety_score}))")
    env.assert_string(f"(disorder (type \"depression\") (score {depression_score}))")
    env.assert_string(f"(yes-count (value {total_yes_count}))")

    # Run the CLIPS environment to process the rules
    env.run()

    # Clear the existing text in the report box
    report_box.configure(state='normal')
    report_box.delete("1.0", tk.END)

    # Initialize default stress level
    stress_level = "Unknown"  # Default stress level

    # Display the results for both assessments in the report box
    report_text = "Your Mental Health Report:\n\n"
    # Initialize a variable to store the result of disorder assessments
    disorder_assessment_result = None

    # Retrieve facts from the CLIPS environment to form the report
    for fact in env.facts():
        if fact.template.name == 'disorder':
            score = fact['score']
            report_text += f"Your {fact['type']} disorder score is: {score}\n"
        elif fact.template.name == 'stress-level':
            stress_level = fact['level']
        elif fact.template.name == 'disorder-assessment':
            # Capture the disorder assessment result
            disorder_assessment_result = fact['result']

    # Check if an assessment result was found, if not it means there are no significant symptoms
    if disorder_assessment_result:
        report_text += f"\n{disorder_assessment_result}"
    else:
        report_text += "\nBased on the symptoms, you are likely in a healthy condition."

    report_text += f"\n\nYour stress level is: {stress_level}"

    report_box.insert(tk.END, report_text)
    report_box.configure(state='disabled')


# Function to reset the input and output
def reset_assessment():
    # Clear all variables and the environment
    for var in eat_disorder_responses.values():
        var.set(0)  # Reset eat disorder responses to "No"
    for var in sleep_disorder_responses.values():
        var.set(0)  # Reset sleep disorder responses to "No"
    for var in anxiety_responses.values():
        var.set(0)  # Reset sleep disorder responses to "No"
    for var in depression_responses.values():
        var.set(0)
    report_box.configure(state='normal')
    report_box.delete("1.0", tk.END)  # Clear the report box
    report_box.configure(state='disabled')


    # Clear the CLIPS environment
    env.clear()

    # Re-build templates and rules
    env.build(eatdisorder_count_template)
    env.build(sleepdisorder_count_template)
    env.build(anxiety_count_template)
    env.build(depression_count_template)
    env.build(yes_count_template)
    env.build(stress_level_template)
    env.build(disorder_template)
    env.build(disorder_assessment_template)  # Add this line
    env.build(maxdisorder_rule)
    env.build(healthy_condition_rule)
    env.build(no_stress_rule)
    env.build(low_stress_rule)
    env.build(moderate_stress_rule)
    env.build(stress_rule)
    env.build(high_stress_rule)

# Clips

In [2]:
# Initialize the CLIPS environment
env = clips.Environment()

# Eating Disorder
eatdisorder_count_template = """
(deftemplate eat-disorder
    (slot score))
"""
env.build(eatdisorder_count_template)

# Sleeping Disorder
sleepdisorder_count_template = """
(deftemplate sleep-disorder
    (slot score))
"""
env.build(sleepdisorder_count_template)

# Anxiety
anxiety_count_template = """
(deftemplate anxiety
    (slot score))
"""
env.build(anxiety_count_template)

# Depression
depression_count_template = """
(deftemplate depression
    (slot score))
"""
env.build(depression_count_template)

disorder_template = """
(deftemplate disorder
    (slot type)
    (slot score))
"""
env.build(disorder_template)

disorder_assessment_template = """
(deftemplate disorder-assessment
    (slot result))
"""
env.build(disorder_assessment_template)

# Define and build the new rules for stress levels
maxdisorder_rule = """
(defrule assess-highest-score
    ?f <- (disorder (type ?type) (score ?score&:(> ?score 2)))
    (not (disorder (score ?higher-score&:(> ?higher-score ?score))))
    =>
    (assert (disorder-assessment (result (str-cat "You may have a " ?type " disorder with a score of " (str-cat ?score) " based on the symptoms. Please refer to a professional.")))))
"""
env.build(maxdisorder_rule)


healthy_condition_rule = """
(defrule assess-healthy-condition
    (not (disorder (score ?score&:(>= ?score 2))))
    =>
    (assert (disorder-assessment (result "Based on the symptoms, you are likely in a healthy condition."))))
"""
env.build(healthy_condition_rule)


# Define and build the yes-count and stress-level templates
yes_count_template = """
(deftemplate yes-count
    (slot value))
"""
env.build(yes_count_template)

stress_level_template = """
(deftemplate stress-level
    (slot level))
"""
env.build(stress_level_template)

no_stress_rule = """
(defrule no-stress
    (yes-count (value ?v&:(<= ?v 0)))
    =>
    (assert (stress-level (level "No Stress"))))
"""
env.build(no_stress_rule)

low_stress_rule = """
(defrule low-stress
    (yes-count (value ?v&:(>= ?v 1)&:(<= ?v 5)))
    =>
    (assert (stress-level (level "Low Stress"))))
"""
env.build(low_stress_rule)

moderate_stress_rule = """
(defrule moderate-stress
    (yes-count (value ?v&:(>= ?v 6)&:(<= ?v 10)))
    =>
    (assert (stress-level (level "Moderate Stress"))))
"""
env.build(moderate_stress_rule)

stress_rule = """
(defrule stress
    (yes-count (value ?v&:(>= ?v 11)&:(<= ?v 15)))
    =>
    (assert (stress-level (level "Stress"))))
"""
env.build(stress_rule)

high_stress_rule = """
(defrule high-stress
    (yes-count (value ?v&:(>= ?v 16)&:(<= ?v 20)))
    =>
    (assert (stress-level (level "High Stress"))))
"""
env.build(high_stress_rule)

# Questions
eat_disorder_questions = [
    "Do you eat very little food? (yes/no): ",
    "Do you spend a lot of time worrying about your weight and body shape? (yes/no): ",
    "Do you make yourself sick or take laxatives after you eat? (yes/no): ",
    "Do you have problems with your digestion, such as bloating, constipation, or diarrhea? (yes/no): ",
    "Do you avoid socializing when you think food will be involved? (yes/no): "
]

sleep_disorder_questions = [
    "Do you have trouble falling asleep and staying asleep? (yes/no): ",
    "Do you sleep for more than 7 hours per day? (yes/no): ",
    "Do you have trouble feeling alert during the day? (yes/no): ",
    "Do you happen to have physical actions or verbal expressions during sleep like walking, talking, or eating? (yes/no): ",
    "Does it usually take more than 30 minutes for you to fall asleep? (yes/no): "
]

anxiety_questions = [
    "Do you tired easily even you have a good rest? (yes/no): ",
    "Do you have difficulty on concentration when doing something? (yes/no): ",
    "Do you feel your hearbeat is always fast? (yes/no): ",
    "Do you have difficulty on breathing or feeling you're choking? (yes/no): ",
    "Do you intensely avoiding on the things you feared or places? (yes/no): "
]

depression_questions = [
    "Have you experienced a persistent lack of interest or pleasure in activities you once enjoyed? (yes/no): ",
    "Do you find yourself frequently overwhelmed by feelings of sadness, hopelessness, or emptiness? (yes/no): ",
    "Have you noticed changes in your sleep patterns, such as difficulty sleeping or sleeping too much? (yes/no): ",
    "Are you struggling with changes in your appetite or weight, whether it's a significant increase or decrease? (yes/no): ",
    "Do you have difficulty concentrating, making decisions, or experiencing feelings of worthlessness or guilt? (yes/no): "
]

# tkinter

In [8]:
import tkinter as tk

def on_mousewheel(event):
    canvas.yview_scroll(int(-1*(event.delta/120)), "units")

def create_section_frame(parent):
    frame = tk.Frame(parent, bg='light blue')
    frame.pack(padx=20, pady=10, fill='x', expand=True)
    return frame

root = tk.Tk()
root.title("Mental Health Assessment")

screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
window_width = screen_width // 2
root.geometry(f'{window_width}x{screen_height}+0+0')
root.configure(bg='light blue')

canvas = tk.Canvas(root, bg='light blue')
scrollbar = tk.Scrollbar(root, orient="vertical", command=canvas.yview)
main_frame = tk.Frame(canvas, bg='light blue')

canvas.configure(yscrollcommand=scrollbar.set)
canvas.bind('<Configure>', lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
canvas.create_window((0, 0), window=main_frame, anchor="nw")

canvas.pack(side="left", fill="both", expand=True)
scrollbar.pack(side="right", fill="y")

# Bind the mousewheel event
root.bind_all("<MouseWheel>", on_mousewheel)

eat_disorder_responses = {}
sleep_disorder_responses = {}
anxiety_responses = {}
depression_responses = {}

# Title label
title_label = tk.Label(main_frame, text="Mental Health Assessment", justify="center", font=("Helvetica", 18, "bold"), bg='light blue')
title_label.pack(pady=20)

# Create a frame for eat disorder questions
eat_disorder_frame = tk.Frame(main_frame)
eat_disorder_frame.pack(padx=10, pady=5, anchor="w")  # Align to the left

# Eating Disorder
eat_disorder_frame = create_section_frame(main_frame)

for question in eat_disorder_questions:
    frame = tk.Frame(eat_disorder_frame)
    frame.pack(fill='x', padx=10, pady=5, anchor="w")  
    
    question_label = tk.Label(frame, text=question, anchor="w", justify="left", width=80) 
    question_label.pack(side=tk.LEFT, fill='x', expand=True, anchor="w")  

    response_var = tk.IntVar(value=0)  # Default to "No"
    eat_disorder_responses[question] = response_var

    yes_button = tk.Radiobutton(frame, text="Yes", variable=response_var, value=1)
    yes_button.pack(side=tk.LEFT)
    no_button = tk.Radiobutton(frame, text="No", variable=response_var, value=0)
    no_button.pack(side=tk.LEFT)

# Create a frame for sleep disorder questions
sleep_disorder_frame = tk.Frame(main_frame)
sleep_disorder_frame.pack(padx=10, pady=5, anchor="w")  # Align to the left

# Sleeping Disorder
sleep_disorder_frame = create_section_frame(main_frame)

for question in sleep_disorder_questions:
    frame = tk.Frame(sleep_disorder_frame)
    frame.pack(fill='x', padx=10, pady=5, anchor="w")  
    
    question_label = tk.Label(frame, text=question, anchor="w", justify="left", width=80) 
    question_label.pack(side=tk.LEFT, fill='x', expand=True, anchor="w")  

    response_var1 = tk.IntVar(value=0)  # Default to "No"
    sleep_disorder_responses[question] = response_var1

    yes_button = tk.Radiobutton(frame, text="Yes", variable=response_var1, value=1)
    yes_button.pack(side=tk.LEFT)
    no_button = tk.Radiobutton(frame, text="No", variable=response_var1, value=0)
    no_button.pack(side=tk.LEFT)

# Anxiety
anxiety_frame = tk.Frame(main_frame)
anxiety_frame.pack(padx=10, pady=5, anchor="w")  # Align to the left
anxiety_frame = create_section_frame(main_frame)

for question in anxiety_questions:
    frame = tk.Frame(anxiety_frame)
    frame.pack(fill='x', padx=10, pady=5, anchor="w")  
    
    question_label = tk.Label(frame, text=question, anchor="w", justify="left", width=80) 
    question_label.pack(side=tk.LEFT, fill='x', expand=True, anchor="w")  

    response_var2 = tk.IntVar(value=0)  # Default to "No"
    anxiety_responses[question] = response_var2

    yes_button = tk.Radiobutton(frame, text="Yes", variable=response_var2, value=1)
    yes_button.pack(side=tk.LEFT)
    no_button = tk.Radiobutton(frame, text="No", variable=response_var2, value=0)
    no_button.pack(side=tk.LEFT)

# Depression
depression_frame = tk.Frame(main_frame)
depression_frame.pack(padx=10, pady=5, anchor="w")  # Align to the left
depression_frame = create_section_frame(main_frame)

for question in depression_questions:
    frame = tk.Frame(depression_frame)
    frame.pack(fill='x', padx=10, pady=5, anchor="w")  
    
    question_label = tk.Label(frame, text=question, anchor="w", justify="left", width=80) 
    question_label.pack(side=tk.LEFT, fill='x', expand=True, anchor="w")  

    response_var3 = tk.IntVar(value=0)  # Default to "No"
    depression_responses[question] = response_var3

    yes_button = tk.Radiobutton(frame, text="Yes", variable=response_var3, value=1)
    yes_button.pack(side=tk.LEFT)
    no_button = tk.Radiobutton(frame, text="No", variable=response_var3, value=0)
    no_button.pack(side=tk.LEFT)

# Create a frame to contain the Submit and Reset buttons
button_frame = tk.Frame(main_frame, bg='light blue')
button_frame.pack(pady=10)

submit_button = tk.Button(main_frame, text="Submit", command=evaluate_and_display, bg='blue', fg='white', font=("Helvetica", 10, "bold"))
submit_button.pack(side=tk.LEFT, padx=10)

reset_button = tk.Button(main_frame, text="Reset", command=reset_assessment, bg='red', fg='white', font=("Helvetica", 10, "bold"))
reset_button.pack(side=tk.LEFT, padx=10)

report_box = tk.Text(main_frame, height=30, width=100) 
report_box.pack(padx=20, pady=20)

root.mainloop()
